In [1]:
import os
import cv2
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from collections import deque
import matplotlib.pyplot as plt
 
from moviepy.editor import *
%matplotlib inline
 
from sklearn.model_selection import train_test_split
 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

In [2]:
# using this parameter makes sure that anyone who re-runs your code will get the exact same outputs.
seed_constant = 27
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [23]:
# frame size
Image_height, Image_width = 64, 64

# specify the number of frames of a video that will be fed to the model as one sequence.
sequence_length = 30

# dataset path
Dataset_dir = "UCF50"

# Activities that will predict
class_list =  ["Basketball", "Biking", "WalkingWithDog", "HorseRace", "PlayingGuitar", "PlayingPiano", "Punch", "Skiing", "GolfSwing"]

In [27]:
# this function will extract the required frames from a video after resizing and normalizing them.
def frames_extraction(video_path):
    
    # list to store frames in video
    frames_list = []

    # read the video
    video_reader = cv2.VideoCapture(video_path)
    
    # total numbers of frame in the video
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # calculate the the interval after which frames will be added to the list
    skip_frames_window = max(int(video_frames_count/sequence_length), 1)
    
    for frame_counter in range(sequence_length):
        
        # set the current frame position of the video
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        
        # reading the frame from the video.
        success, frame = video_reader.read() 
        
        if not success:
            break
 
         # resize the Frame to fixed height and width.
        resized_frame = cv2.resize(frame, (Image_height, Image_width))
        
        #  normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
        
        # append the normalized frame into the frames list
        frames_list.append(normalized_frame)
    
    video_reader.release()

    return frames_list


In [28]:
# this function will extract the data of the selected classes and create the required dataset.

def create_dataset():
   
    # features:          A list containing the extracted frames of the videos.
    # labels:            A list containing the indexes of the classes associated with the videos.
    # video_files_paths: A list containing the paths of the videos in the disk.

 
    # Declared Empty Lists to store the features, labels and video file path values.
    features = []
    labels = []
    video_files_paths = []
    
    # Iterating through all the classes mentioned in the classes list
    for class_index, class_name in enumerate(class_list):
        
        # Display the name of the class whose data is being extracted.
        print(f'Extracting Data of Class: {class_name}')
        
        # Get the list of video files present in the specific class name directory.
        files_list = os.listdir(os.path.join(Dataset_dir, class_name))
        
        # Iterate through all the files present in the files list.
        for file_name in files_list:
            
            # Get the complete video path.
            video_file_path = os.path.join(Dataset_dir, class_name, file_name)
 
            # Extract the frames of the video file.
            frames = frames_extraction(video_file_path)
 
            # Check if the extracted frames are equal to the SEQUENCE_LENGTH specified above.
            # So ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == sequence_length:
 
                # Append the data to their repective lists.
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
 
    # Converting the list to numpy arrays
    features = np.asarray(features)
    labels = np.array(labels)  
    
    # Return the frames, class index, and video file path.
    return features, labels, video_files_paths

In [29]:
# Create the dataset.
features, labels, video_files_paths = create_dataset()

Extracting Data of Class: Basketball
Extracting Data of Class: Biking
Extracting Data of Class: WalkingWithDog
Extracting Data of Class: HorseRace
Extracting Data of Class: PlayingGuitar
Extracting Data of Class: PlayingPiano
Extracting Data of Class: Punch
Extracting Data of Class: Skiing
Extracting Data of Class: GolfSwing


In [30]:
# Using Keras's to_categorical method to convert labels into one-hot-encoded vectors
one_hot_encoded_labels = to_categorical(labels)

In [31]:
# Split the Data into Train ( 75% ) and Test Set ( 25% ).
features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.25, shuffle = True, random_state = seed_constant)

In [36]:
def create_convlstm_model():

    model = Sequential()
    
    model.add(ConvLSTM2D(filters = 4, kernel_size = (3, 3), activation = 'tanh',data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True, input_shape = (sequence_length,
                                                                                      Image_height, Image_width, 3)))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 8, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 14, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(ConvLSTM2D(filters = 16, kernel_size = (3, 3), activation = 'tanh', data_format = "channels_last",
                         recurrent_dropout=0.2, return_sequences=True))
    
    model.add(MaxPooling3D(pool_size=(1, 2, 2), padding='same', data_format='channels_last'))
    #model.add(TimeDistributed(Dropout(0.2)))
    
    model.add(Flatten()) 
    
    model.add(Dense(len(class_list), activation = "softmax"))

    model.summary()
    
    return model

In [37]:
# Construct the required convlstm model.
convlstm_model = create_convlstm_model()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_4 (ConvLSTM2D)  (None, 30, 62, 62, 4)     1024      
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 30, 31, 31, 4)     0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 31, 31, 4)     0         
_________________________________________________________________
conv_lst_m2d_5 (ConvLSTM2D)  (None, 30, 29, 29, 8)     3488      
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 30, 15, 15, 8)     0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 15, 15, 8)     0         
_________________________________________________________________
conv_lst_m2d_6 (ConvLSTM2D)  (None, 30, 13, 13, 14)   

In [40]:
# Create an Instance of Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)
 
# Compile the model and specify loss function, optimizer and metrics values to the model
convlstm_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
 
# Start training the model.
convlstm_model_training_history = convlstm_model.fit(x = features_train, y = labels_train, epochs = 50, batch_size = 4,shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

Epoch 1/50
186/186 [==============================] - 238s 1s/step - loss: 1.7688 - accuracy: 0.3548 - val_loss: 1.1117 - val_accuracy: 0.6203
Epoch 2/50
186/186 [==============================] - 238s 1s/step - loss: 1.1765 - accuracy: 0.5995 - val_loss: 1.1519 - val_accuracy: 0.6310
Epoch 3/50
186/186 [==============================] - ETA: 0s - loss: 0.8949 - accuracy: 0.7016

KeyboardInterrupt: 

In [42]:
# Evaluate the trained model.
model_evaluation_history = convlstm_model.evaluate(features_test, labels_test)

47/47 [==============================] - 9s 182ms/step - loss: 0.7514 - accuracy: 0.7380


In [43]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

date_time_format = '%Y_%m_%d'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
 
# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'convlstm_model_{current_date_time_string}_Loss_{model_evaluation_loss}_Accuracy_{model_evaluation_accuracy}.h5'

# Save your Model.
convlstm_model.save(model_file_name)

In [56]:
def create_LRCN_model():

    # We will use a Sequential model for model construction.
    model = Sequential()
    
    model.add(TimeDistributed(Conv2D(16, (3, 3), padding='same',activation = 'relu'),
                              input_shape = (sequence_length, Image_height, Image_width, 3)))
    
    model.add(TimeDistributed(MaxPooling2D((4, 4)))) 
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(32, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((4, 4))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    model.add(TimeDistributed(Dropout(0.25)))
    
    model.add(TimeDistributed(Conv2D(64, (3, 3), padding='same',activation = 'relu')))
    model.add(TimeDistributed(MaxPooling2D((2, 2))))
    #model.add(TimeDistributed(Dropout(0.25)))
                                      
    model.add(TimeDistributed(Flatten()))
                                      
    model.add(LSTM(32))
                                      
    model.add(Dense(len(class_list), activation = 'softmax'))

    model.summary()
    
    return model

In [57]:
# Construct the required LRCN model.
LRCN_model = create_LRCN_model()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_18 (TimeDis (None, 30, 64, 64, 16)    448       
_________________________________________________________________
time_distributed_19 (TimeDis (None, 30, 16, 16, 16)    0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 30, 16, 16, 16)    0         
_________________________________________________________________
time_distributed_21 (TimeDis (None, 30, 16, 16, 32)    4640      
_________________________________________________________________
time_distributed_22 (TimeDis (None, 30, 4, 4, 32)      0         
_________________________________________________________________
time_distributed_23 (TimeDis (None, 30, 4, 4, 32)      0         
_________________________________________________________________
time_distributed_24 (TimeDis (None, 30, 4, 4, 64)     

In [58]:
# Create an Instance of Early Stopping Callback.
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 15, mode = 'min', restore_best_weights = True)
 
# Compile the model and specify loss function, optimizer and metrics to the model.
LRCN_model.compile(loss = 'categorical_crossentropy', optimizer = 'Adam', metrics = ["accuracy"])
 
# Start training the model.
LRCN_model_training_history = LRCN_model.fit(x = features_train, y = labels_train, epochs = 70, batch_size = 4 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

Epoch 1/70
186/186 [==============================] - 15s 80ms/step - loss: 2.1179 - accuracy: 0.1774 - val_loss: 2.1446 - val_accuracy: 0.1070
Epoch 2/70
186/186 [==============================] - 15s 80ms/step - loss: 1.7820 - accuracy: 0.3441 - val_loss: 1.4304 - val_accuracy: 0.5401
Epoch 3/70
186/186 [==============================] - 14s 77ms/step - loss: 1.5400 - accuracy: 0.4476 - val_loss: 1.2814 - val_accuracy: 0.5668
Epoch 4/70
186/186 [==============================] - 15s 79ms/step - loss: 1.4154 - accuracy: 0.5067 - val_loss: 1.1082 - val_accuracy: 0.6257
Epoch 5/70
186/186 [==============================] - 15s 79ms/step - loss: 1.1460 - accuracy: 0.6142 - val_loss: 0.9421 - val_accuracy: 0.6631
Epoch 6/70
186/186 [==============================] - 15s 79ms/step - loss: 1.0657 - accuracy: 0.6398 - val_loss: 0.7817 - val_accuracy: 0.7380
Epoch 7/70
186/186 [==============================] - 15s 80ms/step - loss: 1.0163 - accuracy: 0.6452 - val_loss: 0.8301 - val_accuracy:

In [59]:
# Evaluate the trained model.
model_evaluation_history = LRCN_model.evaluate(features_test, labels_test)

10/10 [==============================] - 1s 105ms/step - loss: 0.4112 - accuracy: 0.8714


In [61]:
# Get the loss and accuracy from model_evaluation_history.
model_evaluation_loss, model_evaluation_accuracy = model_evaluation_history

date_time_format = '%Y_%m_%d'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)
    
# Define a useful name for our model to make it easy for us while navigating through multiple saved models.
model_file_name = f'LRCN_model_Date_Time_{current_date_time_string}_Loss_{model_evaluation_loss}_Accuracy_{model_evaluation_accuracy}.h5'
 
# Save the Model.
LRCN_model.save(model_file_name)

In [47]:
from tensorflow import keras

# load model
LRCN_model = keras.models.load_model("convlstm_model_2022_05_01_Loss_0.7514086365699768_Accuracy_0.7379679083824158.h5")

In [67]:
def predict_single_action(video_file_path, sequence_length):

    # Initialize the VideoCapture object to read from the video file.
    video_reader = cv2.VideoCapture(video_file_path)

    # Get the width and height of the video.
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Declare a list to store video frames we will extract.
    frames_list = []
    
    # Initialize a variable to store the predicted action being performed in the video.
    predicted_class_name = ''

    # Get the number of frames in the video.
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    # Calculate the interval after which frames will be added to the list.
    skip_frames_window = max(int(video_frames_count/sequence_length),1)

    # Iterating the number of times equal to the fixed length of sequence.
    for frame_counter in range(sequence_length):

        # Set the current frame position of the video.
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

        # Read a frame.
        success, frame = video_reader.read() 

        # Check if frame is not read properly then break the loop.
        if not success:
            break

        # Resize the Frame to fixed Dimensions.
        resized_frame = cv2.resize(frame, (Image_height, Image_width))
        
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
        normalized_frame = resized_frame / 255
        
        # Appending the pre-processed frame into the frames list
        frames_list.append(normalized_frame)

    # Passing the  pre-processed frames to the model and get the predicted probabilities.
    predicted_labels_probabilities = LRCN_model.predict(np.expand_dims(frames_list, axis = 0))[0]

    # Get the index of class with highest probability.
    predicted_label = np.argmax(predicted_labels_probabilities)

    # Get the class name using the retrieved index.
    predicted_class_name = class_list[predicted_label]
    
    # Display the predicted action along with the prediction confidence.
    if predicted_labels_probabilities[predicted_label] > 0.5:
        print(f'Action Predicted: {predicted_class_name}\nConfidence: {predicted_labels_probabilities[predicted_label]}')
    else:
        print("Couldn't detected")
        
    # Release the VideoCapture object. 
    video_reader.release()

In [73]:
input_video_file_path = f'test_videos/go.mp4'
 
# Perform Single Prediction on the Test Video.
predict_single_action(input_video_file_path, sequence_length)

Couldn't detected
